In [11]:
# import pandas as pd
import os

from sklearn.linear_model import LinearRegression
import numpy as np

import warnings
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX

import pandas as pd
from datetime import datetime, timedelta

from sklearn.linear_model import LinearRegression
import numpy as np
import warnings
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime, timedelta

from fpdf import FPDF
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.platypus import Table, TableStyle, SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet


# Function to load financial data from CSV, or create a new empty DataFrame if the file doesn't exist
def load_financial_data():
    if os.path.exists('financial_data.csv'):
        return pd.read_csv('financial_data.csv')
    else:
        return pd.DataFrame(columns=['date', 'product', 'supplier', 'labor_cost', 'material_cost', 'tax', 'profit', 
                                     'labor_ratio', 'material_ratio', 'total_revenue', 'expenses', 'net_profit'])

# Function to load sales data from CSV, or create a new empty DataFrame if the file doesn't exist
def load_sales_data():
    if os.path.exists('sales_data.csv'):
        return pd.read_csv('sales_data.csv')
    else:
        return pd.DataFrame(columns=['date', 'product', 'quantity_sold', 'price', 'labor_cost', 'material_cost', 
                                     'discount', 'total_revenue', 'month'])

# Function to load inventory data from CSV, or create a new empty DataFrame if the file doesn't exist
def load_inventory_data():
    if os.path.exists('inventory.csv'):
        return pd.read_csv('inventory.csv')
    else:
        return pd.DataFrame(columns=['product', 'stock_level', 'reorder_level', 'unit_cost', 'supplier_name'])

# Function to validate date format (YYYY-MM-DD)
def validate_date_format(date):
    try:
        pd.to_datetime(date, format='%Y-%m-%d')
        return True
    except ValueError:
        return False

# Function to validate month format (YYYY-MM)
def validate_month_format(month):
    try:
        pd.to_datetime(month, format='%Y-%m')
        return True
    except ValueError:
        return False

# Function to validate if the product exists in the inventory or sales data
def validate_product_name(product):
    inventory_data = load_inventory_data()
    sales_data = load_sales_data()
    # Check if the product exists in either the inventory or sales data
    if product in inventory_data['product'].values or product in sales_data['product'].values:
        return True
    return False

# Function to load supplier data from CSV
def load_supplier_data():
    if os.path.exists('supplier_data.csv'):
        return pd.read_csv('supplier_data.csv')
    else:
        return pd.DataFrame(columns=['supplier_name', 'product', 'cost', 'delivery_time'])

from sklearn.linear_model import LinearRegression
import numpy as np

# Function to suggest optimal price for maximum profit
def suggest_optimal_price():
    sales_data = load_sales_data()
    
    product = input("Enter the product name: ")
    if product not in sales_data['product'].values:
        print(f"Product '{product}' not found in sales data.")
        return
    
    # Filter sales data for selected product
    product_data = sales_data[sales_data['product'] == product]
    
    if product_data.shape[0] < 5:
        print("Not enough sales data available for reliable prediction. At least 5 records are recommended.")
        return

    # Group by price and sum quantity sold to handle multiple entries
    grouped = product_data.groupby('price')['quantity_sold'].sum().reset_index()
    
    # Prepare features and target
    X = grouped[['price']].values
    y = grouped['quantity_sold'].values

    # Train linear regression model
    model = LinearRegression()
    model.fit(X, y)

    # Simulate prices over a reasonable range
    min_price = grouped['price'].min()
    max_price = grouped['price'].max()
    price_range = np.linspace(min_price, max_price, 100).reshape(-1, 1)
    
    # Predict quantity for each price and calculate revenue
    predicted_quantity = model.predict(price_range)
    predicted_revenue = price_range.flatten() * predicted_quantity
    
    # Find the price with max revenue
    optimal_index = np.argmax(predicted_revenue)
    optimal_price = round(price_range[optimal_index][0], 2)
    max_revenue = round(predicted_revenue[optimal_index], 2)

    print("\nOptimal Price Suggestion:")
    print(f"Product: {product}")
    print(f"Suggested Price: ${optimal_price}")
    print(f"Expected Revenue: ${max_revenue}")
    

def suggest_best_supplier():
    inventory = load_inventory_data()
    supplier_data = load_supplier_data()

    product = input("Enter the product name: ")
    if product not in inventory['product'].values:
        print(f"Product '{product}' not found in inventory.")
        return

    # Merge inventory with supplier data
    merged = pd.merge(inventory, supplier_data, left_on=['product', 'supplier_name'], right_on=['product', 'supplier'])

    # Filter for selected product only
    product_suppliers = merged[merged['product'] == product]

    if product_suppliers.empty:
        print(f"No supplier data found for product '{product}'.")
        return
        
    # Create a copy to avoid the warning
    product_suppliers = product_suppliers.copy()

    # Compute score: lower is better
    product_suppliers.loc[:, 'score'] = product_suppliers['delivery_time'] + 0.01 * product_suppliers['cost']

    # Sort by score
    best_supplier = product_suppliers.sort_values(by='score').iloc[0]

    print("\nBest Supplier Suggestion:")
    print(f"Product: {best_supplier['product']}")
    print(f"Supplier: {best_supplier['supplier_name']}")
    print(f"Cost: ${best_supplier['cost']}")
    print(f"Delivery Time: {best_supplier['delivery_time']} days")
    print(f"Score: {round(best_supplier['score'], 2)}")

def load_inventory_data():
    return pd.read_csv('inventory.csv')

def load_sales_data():
    return pd.read_csv('sales_data.csv')

def show_optimal_restocking_times():
    inventory = load_inventory_data()
    sales = load_sales_data()

    if inventory.empty or sales.empty:
        print("Insufficient inventory or sales data to calculate restocking times.")
        return

    sales['date'] = pd.to_datetime(sales['date'])
    restock_forecasts = []
    below_reorder_products = []

    for product in inventory['product'].unique():
        product_sales = sales[sales['product'] == product]

        if product_sales.empty:
            continue

        # Calculate daily average sales
        days_range = (product_sales['date'].max() - product_sales['date'].min()).days
        if days_range == 0:
            continue  # Avoid division by zero
        total_quantity = product_sales['quantity_sold'].sum()
        avg_daily_sales = total_quantity / days_range

        product_inventory = inventory[inventory['product'] == product]
        for _, row in product_inventory.iterrows():
            stock = row['stock_level']
            reorder_level = row['reorder_level']
            supplier = row['supplier_name']

            if stock < reorder_level:
                below_reorder_products.append({
                    'product': product,
                    'supplier': supplier,
                    'current_stock': stock,
                    'reorder_level': reorder_level
                })

            if avg_daily_sales > 0 and stock >= reorder_level:
                days_until_reorder = (stock - reorder_level) / avg_daily_sales
                reorder_date = datetime.now() + timedelta(days=days_until_reorder)
                restock_forecasts.append({
                    'product': product,
                    'supplier': supplier,
                    'current_stock': stock,
                    'reorder_level': reorder_level,
                    'avg_daily_sales': round(avg_daily_sales, 2),
                    'estimated_reorder_date': reorder_date.strftime('%Y-%m-%d')
                })

    if below_reorder_products:
        df_below = pd.DataFrame(below_reorder_products)
        print("\nProducts Below Reorder Level:")
        print(df_below.to_string(index=False))
    else:
        print("\nNo products are currently below the reorder level.")

    if restock_forecasts:
        df_forecasts = pd.DataFrame(restock_forecasts)
        print("\nOptimal Restocking Times (for products above reorder level):")
        print(df_forecasts.to_string(index=False))
    else:
        print("\nNo restocking forecasts available for products currently above reorder level.")


# Function to update the sales data
def update_sales_data():
    sales_data = load_sales_data()

    # Get user input for sales details with validation
    date = input("Enter the date (YYYY-MM-DD): ")
    while not validate_date_format(date):
        print("Invalid date format. Please enter the date in the correct format (YYYY-MM-DD).")
        date = input("Enter the date (YYYY-MM-DD): ")

    product = input("Enter the product name: ")
    while not validate_product_name(product):
        print(f"Invalid product name. Please enter a valid product name from the inventory or sales data.")
        product = input("Enter the product name: ")

    try:
        quantity_sold = int(input("Enter the quantity sold: "))
    except ValueError:
        print("Invalid input for quantity sold. Please enter an integer.")
        return

    try:
        price = float(input("Enter the price of the product: "))
        price = round(price, 2)  # Round to two decimal places
    except ValueError:
        print("Invalid input for price. Please enter a valid number.")
        return

    try:
        labor_cost = float(input("Enter the labor cost: "))
        labor_cost = round(labor_cost, 2)  # Round to two decimal places
    except ValueError:
        print("Invalid input for labor cost. Please enter a valid number.")
        return

    try:
        material_cost = float(input("Enter the material cost: "))
        material_cost = round(material_cost, 2)  # Round to two decimal places
    except ValueError:
        print("Invalid input for material cost. Please enter a valid number.")
        return

    try:
        discount = float(input("Enter the discount (as a percentage): "))
        discount = round(discount, 2)  # Round to two decimal places
    except ValueError:
        print("Invalid input for discount. Please enter a valid number.")
        return

    # Calculate total revenue considering the discount
    total_revenue = quantity_sold * price * (1 - discount / 100)
    
    month = input("Enter the month (YYYY-MM): ")
    while not validate_month_format(month):
        print("Invalid month format. Please enter the month in the correct format (YYYY-MM).")
        month = input("Enter the month (YYYY-MM): ")
    
    # Extract the month from the date and ensure consistency
    date_month = date[:7]  # Get the YYYY-MM from the date
    if month != date_month:
        print(f"Month from date ({date_month}) does not match the entered month ({month}).")
        print(f"Updating month to match the date: {date_month}.")
        month = date_month

    # Create a new row for sales data
    new_row = pd.DataFrame([{
        'date': date,
        'product': product,
        'quantity_sold': quantity_sold,
        'price': price,
        'labor_cost': labor_cost,
        'material_cost': material_cost,
        'discount': discount,
        'total_revenue': total_revenue,
        'month': month
    }])

    # Append the new row to the existing sales data
    sales_data = pd.concat([sales_data, new_row], ignore_index=True)

    # Save the updated sales data to the CSV file
    sales_data.to_csv('sales_data.csv', index=False)

    print("Sales data updated successfully.")

# Function to update the financial data
def update_financial_data():
    financial_data = load_financial_data()
    inventory_data = load_inventory_data()
    date = input("Enter the date (YYYY-MM-DD): ")
    while not validate_date_format(date):
        print("Invalid date format. Please enter the date in the correct format (YYYY-MM-DD).")
        date = input("Enter the date (YYYY-MM-DD): ")
    
    product = input("Enter the product name: ")
    while not validate_product_name(product):
        print(f"Invalid product name. Please enter a valid product name from the inventory or sales data.")
        product = input("Enter the product name: ")

    supplier = input("Enter the supplier name: ")
    product_inventory = inventory_data[inventory_data['product'] == product]
    if product_inventory.empty:
        print(f"No inventory data found for product: {product}")
        return
    labor_cost = float(input("Enter the labor cost: "))
    labor_cost = round(labor_cost, 2)
    material_cost = float(input("Enter the material cost: "))
    material_cost = round(material_cost, 2)
    tax_rate = float(input("Enter the tax rate (as a percentage): "))
    tax_rate = round(tax_rate, 2)
    total_revenue = float(input("Enter the total revenue: "))
    total_revenue = round(total_revenue, 2)
    tax = total_revenue * (tax_rate / 100)
    profit = total_revenue - (labor_cost + material_cost + tax)
    labor_ratio = labor_cost / total_revenue
    material_ratio = material_cost / total_revenue
    expenses = labor_cost + material_cost + tax
    net_profit = profit - expenses
    new_row = pd.DataFrame([{
        'date': date,
        'product': product,
        'supplier': supplier,
        'labor_cost': labor_cost,
        'material_cost': material_cost,
        'tax': tax,
        'profit': profit,
        'labor_ratio': labor_ratio,
        'material_ratio': material_ratio,
        'total_revenue': total_revenue,
        'expenses': expenses,
        'net_profit': net_profit
    }])
    financial_data = pd.concat([financial_data, new_row], ignore_index=True)
    financial_data.to_csv('financial_data.csv', index=False)
    print("Financial data updated successfully.")

def view_financial_data():
    try:
        sales_data = pd.read_csv('sales_data.csv')
        sales_data['month'] = pd.to_datetime(sales_data['month'], errors='coerce')  # Convert to datetime
        grouped = sales_data.groupby(sales_data['month'].dt.to_period("M"))

        tax_rate = 0.07  # 7% tax

        financial_records = []

        for month, group in grouped:
            total_revenue = group['total_revenue'].sum()
            labor_cost = group['labor_cost'].sum()
            material_cost = group['material_cost'].sum()
            expenses = labor_cost + material_cost
            tax = total_revenue * tax_rate
            profit = total_revenue - expenses - tax
            labor_ratio = labor_cost / total_revenue if total_revenue != 0 else 0
            material_ratio = material_cost / total_revenue if total_revenue != 0 else 0
            net_profit = profit

            financial_records.append({
                'date': str(month),
                'labor_cost': round(labor_cost, 2),
                'material_cost': round(material_cost, 2),
                'tax': round(tax, 2),
                'profit': round(profit, 2),
                'labor_ratio': round(labor_ratio, 2),
                'material_ratio': round(material_ratio, 2),
                'total_revenue': round(total_revenue, 2),
                'expenses': round(expenses, 2),
                'net_profit': round(net_profit, 2)
            })

        financial_df = pd.DataFrame(financial_records)
        financial_df.to_csv('financial_data.csv', index=False)
        print("Financial data updated and saved to financial_data.csv.\n")
        print(financial_df.to_string(index=False))  # Ensures one row per line, no index

    except FileNotFoundError:
        print("sales_data.csv not found.")
    except Exception as e:
        print(f"An error occurred: {e}")



# Function to update the inventory data based on current stock level
def update_inventory_data():
    inventory_data = load_inventory_data()
    product = input("Enter the product name: ")
    while not validate_product_name(product):
        print(f"Invalid product name. Please enter a valid product name from the inventory or sales data.")
        product = input("Enter the product name: ")
    
    supplier_name = input("Enter the supplier name: ")
    product_inventory = inventory_data[(inventory_data['product'] == product) & 
                                       (inventory_data['supplier_name'] == supplier_name)]
    if not product_inventory.empty:
        current_stock = int(input(f"Enter the current stock level for {product}: "))
        inventory_data.loc[(inventory_data['product'] == product) & 
                           (inventory_data['supplier_name'] == supplier_name), 'stock_level'] = current_stock
        reorder_level = product_inventory.iloc[0].get('reorder_level', None)
        if pd.notnull(reorder_level):
            if current_stock < reorder_level:
                print("Warning: Stock is below the reorder level. You need to restock.")
        else:
            print("Note: No reorder level specified for this product.")
        print(f"Inventory updated: {product} (Supplier: {supplier_name}) - New Stock Level: {current_stock}")
    else:
        print(f"No existing inventory found for product '{product}' from supplier '{supplier_name}'.")
    inventory_data.to_csv('inventory.csv', index=False)
    print("Inventory data updated successfully.")

# Function to view the inventory data (includes stock_level, reorder_level, unit_cost, supplier_name)
def view_inventory_data():
    inventory_data = load_inventory_data()
    if inventory_data.empty:
        print("No inventory data available.")
    else:
        columns_to_display = ['product', 'stock_level', 'reorder_level', 'unit_cost', 'supplier_name']
        inventory_data_to_display = inventory_data[columns_to_display]
        pd.set_option('display.width', None)
        pd.set_option('display.max_columns', None)
        print("\nInventory Data:")
        print(inventory_data_to_display.to_string(index=False))

def generate_pdf_report(filename):
    financial_data = load_financial_data()
    sales_data = load_sales_data()
    inventory_data = load_inventory_data()
    supplier_data = load_supplier_data()  # Assuming a function to load the supplier data

    pdf_file = f"{filename}.pdf"
    doc = SimpleDocTemplate(pdf_file, pagesize=letter)
    styles = getSampleStyleSheet()
    flowables = []

    # Add a title for the whole report
    title = Paragraph("<b>Financial Data & Breakdown</b>", styles['Heading1'])
    flowables.append(title)
    flowables.append(Spacer(1, 20))

    # Function to add a table to the PDF
    def add_table(df, title, is_financial=False):
        flowables.append(Paragraph(f"<b>{title}</b>", styles['Heading2']))
        if df.empty:
            flowables.append(Paragraph("No data available.", styles['Normal']))
        else:
            data = [df.columns.tolist()] + df.values.tolist()
            table = Table(data, repeatRows=1)

            # Table styling (font size adjusted for financial data)
            if is_financial:
                table.setStyle(TableStyle([
                    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                    ('FONTSIZE', (0, 0), (-1, -1), 8),  # Reduced font size for financial data
                    ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
                    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                    ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
                ]))
            else:
                table.setStyle(TableStyle([
                    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                    ('FONTSIZE', (0, 0), (-1, 0), 10),
                    ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
                    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                    ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
                ]))

            flowables.append(table)
        flowables.append(Spacer(1, 20))

    # Add tables to the report in the desired order
    add_table(financial_data, "Financial Data", is_financial=True)  # Financial data first
    add_table(inventory_data, "Inventory Data")
    add_table(supplier_data, "Supplier Data")  # Supplier data after inventory
    add_table(sales_data, "Sales Data")

    # Generate the PDF
    doc.build(flowables)
    print(f"\n PDF report generated and saved as: {pdf_file}")


def main():
    while True:
        print("\nMenu:")
        print("1. Update Sales Data")
        print("2. View Financial Data")
        print("3. Update Inventory Data")
        print("4. View Inventory Data")
        print("5. Find Optimal Supplier for Product")
        print("6. Calculate Optimal Selling Price")
        print("7. View Restocking Times")
        print("8. Generate PDF Report")
        print("9. Exit")

        choice = input("Enter your choice: ")

        if choice == '1':
            update_sales_data()
        elif choice == '2':
            view_financial_data()
        elif choice == '3':
            update_inventory_data()
        elif choice == '4':
            view_inventory_data()
        elif choice == '5':
            suggest_best_supplier()
        elif choice == '6':
            suggest_optimal_price()
        elif choice == '7':
            show_optimal_restocking_times()
        elif choice == '8':
            filename = input("Enter the filename to save the PDF (e.g., 'report.pdf'): ")
            generate_pdf_report(filename)
        elif choice == '9':
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 10.")

if __name__ == "__main__":
    main()


Menu:
1. Update Sales Data
2. View Financial Data
3. Update Inventory Data
4. View Inventory Data
5. Find Optimal Supplier for Product
6. Calculate Optimal Selling Price
7. View Restocking Times
8. Generate PDF Report
9. Exit


Enter your choice:  2


Financial data updated and saved to financial_data.csv.

   date  labor_cost  material_cost     tax   profit  labor_ratio  material_ratio  total_revenue  expenses  net_profit
2025-01      487.81        1138.16 1272.03 15273.91         0.03            0.06       18171.91   1625.97    15273.91
2025-02      523.73        1222.07 1265.08 15061.67         0.03            0.07       18072.55   1745.80    15061.67
2025-03      802.63        1872.81 2113.20 25399.88         0.03            0.06       30188.52   2675.44    25399.88
2025-04      563.89        1315.68 1456.04 17464.90         0.03            0.06       20800.50   1879.57    17464.90

Menu:
1. Update Sales Data
2. View Financial Data
3. Update Inventory Data
4. View Inventory Data
5. Find Optimal Supplier for Product
6. Calculate Optimal Selling Price
7. View Restocking Times
8. Generate PDF Report
9. Exit


Enter your choice:  4



Inventory Data:
 product  stock_level  reorder_level  unit_cost  supplier_name
Widget A           60             48      46.18   Beta Traders
Widget B          132             58      27.98   Beta Traders
Gadget C           65             48      12.35      Gamma Co.
Gadget D           71             52      35.14 Alpha Supplies

Menu:
1. Update Sales Data
2. View Financial Data
3. Update Inventory Data
4. View Inventory Data
5. Find Optimal Supplier for Product
6. Calculate Optimal Selling Price
7. View Restocking Times
8. Generate PDF Report
9. Exit


...


Inventory Data:
 product  stock_level  reorder_level  unit_cost  supplier_name
Widget A           60             48      46.18   Beta Traders
Widget B          132             58      27.98   Beta Traders
Gadget C           20             48      12.35      Gamma Co.
Gadget D           71             52      35.14 Alpha Supplies

Menu:
1. Update Sales Data
2. View Financial Data
3. Update Inventory Data
4. View Inventory Data
5. Find Optimal Supplier for Product
6. Calculate Optimal Selling Price
7. View Restocking Times
8. Generate PDF Report
9. Exit


Enter your choice:  7



Products Below Reorder Level:
 product  supplier  current_stock  reorder_level
Gadget C Gamma Co.             20             48

Optimal Restocking Times (for products above reorder level):
 product       supplier  current_stock  reorder_level  avg_daily_sales estimated_reorder_date
Widget A   Beta Traders             60             48             3.39             2025-05-08
Widget B   Beta Traders            132             58             3.76             2025-05-24
Gadget D Alpha Supplies             71             52             4.35             2025-05-08

Menu:
1. Update Sales Data
2. View Financial Data
3. Update Inventory Data
4. View Inventory Data
5. Find Optimal Supplier for Product
6. Calculate Optimal Selling Price
7. View Restocking Times
8. Generate PDF Report
9. Exit


Enter your choice:  9


Exiting the program.
